# Minimum Spanning Tree (MST) / Árvore Geradora Mínima (AGM)

```
ALGORITMO-GENERICO(G, w):
    A = 0
    while A não é uma AGM:
        ache uma aresta (u, v) que é segura para A
        A = A U {(u, v)} 
    return A
```

O que vai mudar de algoritmo para algoritmo é a maneira como a aresta segura é encontrada.

## KRUSKAL

- Usa uma floresta de componentes, e basicamente vai conectando os componentes usando arestas seguras.
- A estrutura de dados usada é um DISJOINT-SET
- Cada SET contém os vértices em uma árvore da floresta atual. 
- A operação FIND-SET(u) retorna um elemento representativo do SET que contem u.
- Portanto, para determinar se dois vértices u e v pertencem a mesma árvore, apenas testa se FIND-SET(u) é igual ao FIND-SET(v).
- Para combinar as árvores, o algoritmo chama UNION.

### Criando a estrutura DISJOINT_SET

In [184]:
class DisjointSet:
    def __init__(self):
        self.parent = []
        self.rank = []
    
    # make a and b part of the same component
    # union by rank optimization
    def union(self, a, b):
        pa = self.find(a)
        pb = self.find(b)
        if pa == pb: return
        if self.rank[pa] > self.rank[pb]:
            self.parent[pb] = pa
            self.rank[pa] += self.rank[pb]
        else:
            self.parent[pa] = pb
            self.rank[pb] += self.rank[pa]
    
    # find the representative of the 
    # path compression optimization
    def find(self, a):
        if self.parent[a] == a:
            return a
        
        self.parent[a] = self.find(self.parent[a])
        return self.parent[a]

### Criando o grafo

In [124]:
"""Same as before however added method for listing edges"""

class Node:  
    def __init__(self, name, color=None, d=None, pi=None):
        self.name = name
        self.adj_list = []
        self.color = color  # estado: visitado, não visitado, explorado
        self.d = d  # distancia do vértice origem
        self.pi = pi  # vértice predecessor

class Graph:
    def __init__(self, num_vertices=None, undirected=True):
        self.num_vertices = num_vertices
        self.nodes = [Node(v) for v in range(num_vertices)]
        self.undirected = undirected
               
    def add_edge(self, source, dest, weight):
        self.nodes[source].adj_list.append((self.nodes[dest], weight))
        if self.undirected:
            self.nodes[dest].adj_list.append((self.nodes[source], weight))
        
    def print(self):
        for node in self.nodes:
            print(f"Node {node.name} -> {[(n.name, w) for n, w in node.adj_list]}")
            
    def get_list_edges(self):
        edges = []
        for u in self.nodes:
            for v, w in u.adj_list:
                if self.undirected:  # So it won't repeat edges
                    first = min(u.name, v.name)
                    second = v.name if first == u.name else u.name
                    edge = (first, second, w)
                    edges.append(edge) if edge not in edges else None
                else:
                    edges.append((u.name, v.name, w))

        return edges
    
    def get_sorted_list_edges(self):
        edges = self.get_list_edges()
        return sorted(edges, key=lambda tup: tup[2])


<img src="./img/weighet_graph.png" alt="drawing" width="400"/>

In [188]:
g = Graph(5)
g.add_edge(0, 1, 3)
g.add_edge(0, 3, 7)
g.add_edge(0, 4, 8)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 4)
g.add_edge(2, 3, 2)
g.add_edge(3, 4, 3)
g.print()

Node 0 -> [(1, 3), (3, 7), (4, 8)]
Node 1 -> [(0, 3), (2, 1), (3, 4)]
Node 2 -> [(1, 1), (3, 2)]
Node 3 -> [(0, 7), (1, 4), (2, 2), (4, 3)]
Node 4 -> [(0, 8), (3, 3)]


In [189]:
g.get_list_edges()

[(0, 1, 3), (0, 3, 7), (0, 4, 8), (1, 2, 1), (1, 3, 4), (2, 3, 2), (3, 4, 3)]

In [190]:
g.get_sorted_list_edges()

[(1, 2, 1), (2, 3, 2), (0, 1, 3), (3, 4, 3), (1, 3, 4), (0, 3, 7), (0, 4, 8)]

## Algoritmo

ref: https://www.geeksforgeeks.org/kruskals-minimum-spanning-tree-algorithm-greedy-algo-2/

In [191]:
def MST_Kruskall(G):
    A = set()

    DS = DisjointSet()
    for v in G.nodes:
        DS.parent.append(v.name)
        DS.rank.append(0)

    min_cost = 0
    edges = G.get_list_edges()
    ordered_edges = G.get_sorted_list_edges()
    for u, v, w in ordered_edges:
        if DS.find(u) != DS.find(v):
            A = A.union((u, v))
            DS.union(u, v)
            min_cost += w
            print(f"{u} -- {v} == {w}")
    print("Minimum Spanning Tree:", min_cost)
    return A

In [193]:
MST_Kruskall(g)

1 -- 2 == 1
2 -- 3 == 2
0 -- 1 == 3
3 -- 4 == 3
Minimum Spanning Tree: 9


{0, 1, 2, 3, 4}

In [195]:
g2 = Graph(5)
g2.add_edge(0, 1, 12)
g2.add_edge(0, 3, 7)
g2.add_edge(0, 4, 8)
g2.add_edge(1, 2, 1)
g2.add_edge(1, 3, 4)
g2.add_edge(2, 3, 2)
g2.add_edge(3, 4, 3)

MST_Kruskall(g2)

1 -- 2 == 1
2 -- 3 == 2
3 -- 4 == 3
0 -- 3 == 7
Minimum Spanning Tree: 13


{0, 1, 2, 3, 4}